In [ ]:
%matplotlib inline
from deepdecoder.networks import dcgan_generator, dcgan_discriminator
from beras.gan import GAN
from beras.models import asgraph
import matplotlib.pyplot as plt
import numpy as np
from deepdecoder.mask_loss import mask_loss, mask_loss_sobel, mask_loss_mse, mask_loss_adaptive_mse
from deepdecoder.utils import binary_mask, visualise_tiles
from deepdecoder.data import grids_lecture_generator, load_real_hdf5_tags
from deepdecoder.visualise import plt_hist
from beesgrid import MASK
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import Callback
from keras.objectives import mse
import seaborn as sns
import matplotlib
import time
from itertools import count

import pylab
pylab.rcParams['figure.figsize'] = (18, 18)

In [ ]:
batch_size = 128
n = 32
generator_input_dim = 100
num_batches_per_epoch = 100
epoch_size = batch_size*num_batches_per_epoch
z_shape = (batch_size, generator_input_dim)

In [ ]:
g = dcgan_generator(n, input_dim=generator_input_dim)

In [ ]:
def self_similarity(zs, imgs):
    img_deltas = (imgs - imgs[::-1])**2
    z_delta = (zs - zs[::-1])**2
    return ((img_deltas.mean() - z_delta.mean())**2).mean()

In [ ]:
g.compile(Adam(lr=0.0001, beta_1=0.5), self_similarity)

In [ ]:
def z_vec():
    return np.random.uniform(-1, 1, z_shape)
    
def z_generator():
    while True:
        z = z_vec()
        yield [z], z.reshape(z_shape[0], 1, 1, z_shape[1])

In [ ]:
class Visualise(Callback):
    def on_epoch_end(self, epoch, log={}):
        plt.grid(False)
        visualise_tiles(g.predict(z_vec()))

In [ ]:
g.fit_generator(z_generator(), samples_per_epoch=epoch_size, nb_epoch=10, verbose=1, callbacks=[Visualise()])